In [5]:
import pandas as pd
from scipy.optimize import fsolve
from scipy.stats import norm
import numpy as np

final_data= pd.read_csv('merged_data.csv')

In [6]:
# DD 값이 0 이하인 경우 Default를 1로 설정, 그 외는 0으로 설정
final_data['Default'] = (final_data['DD'] <= 0).astype(int)

# Step 1: permno를 기준으로 final_data를 그룹화
grouped_data = final_data.groupby('permno')

# Step 2: 각 그룹 내에서 Default 값을 전 년도로 시프트
# shift(1)은 이전 행의 값을 현재 행으로 이동시키는 함수
final_data['y'] = grouped_data['Default'].shift(1)






In [7]:
# Quick Ratio
final_data['quick_ratio'] = (final_data['act'] - final_data['invt']) / final_data['lct'].replace(0, np.nan)

# Current Ratio
final_data['current_ratio'] = final_data['act'] / final_data['lct'].replace(0, np.nan)

# Accounts Receivable Turnover
final_data['art'] = final_data['sale'] / final_data['rect'].replace(0, np.nan)

# EBITDA
final_data['ebitda'] = final_data['oiadp'] + final_data['dp']

# Inventory Turnover
final_data['inventory_turnover'] = final_data['cogs'] / final_data['invt'].replace(0, np.nan)

# Return on Equity (ROE)
final_data['roe'] = final_data['ni'] / (final_data['at'] - final_data['dltt'] - final_data['dlc']).replace(0, np.nan)

# Return on Assets (ROA)
final_data['roa'] = final_data['ni'] / final_data['at'].replace(0, np.nan)

# Gross Margin
final_data['gross_margin'] = (final_data['sale'] - final_data['cogs']) / final_data['sale'].replace(0, np.nan)

# Cost of Sales Ratio
final_data['cost_of_sales_ratio'] = final_data['cogs'] / final_data['sale'].replace(0, np.nan)

# Net Profit Growth Rate YoY (add epsilon to avoid division by zero)
final_data = final_data.sort_values(by=['gvkey', 'datadate'])
final_data['net_profit_growth'] = final_data.groupby('gvkey')['ni'].pct_change().replace([np.inf, -np.inf], np.nan)

# Revenue Growth Rate YoY (add epsilon to avoid division by zero)
final_data['revenue_growth'] = final_data.groupby('gvkey')['sale'].pct_change().replace([np.inf, -np.inf], np.nan)

# Debt to Asset Ratio
final_data['debt_to_asset'] = (final_data['dltt'] + final_data['dlc']) / final_data['at'].replace(0, np.nan)

# Handling infinities and missing values
final_data.replace([np.inf, -np.inf], np.nan, inplace=True)
final_data.dropna(inplace=True)

# Drop unnecessary columns after processing
final_data = final_data.drop(columns=['act', 'invt', 'lct', 'sale', 'rect', 'oiadp', 'dp', 'cogs', 'ni', 'at', 'dltt', 'dlc','permno','cusip','date','shrout','prc','year','month','gvkey','datadate','sigma_E','DD','Default','che','ib','rf'])


In [8]:
final_data.to_csv('Data_AfterProcessing.csv', index=False)